**Learning outcomes**

* Describe the strategy of amplitude amplification.
* Define the diffusion operator and visualize its effect on the uniform superposition.

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

**Codercise G.1.1.**
 Complete the following code for returning the amplitudes after applying the oracle to the uniform superposition. The oracle is accessible as oracle_matrix(combo), where combo is the secret combination. Amplitudes will be plotted for combo = [0, 0, 0, 1].

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution
    my_array = np.identity(2**len(combo)) # Create the identity matrix
    my_array[index, index] = -1
    return my_array

@qml.qnode(dev)
def oracle_amp(combo):
    """Prepare the uniform superposition and apply the oracle.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle.
    """
    ## superposition
    qml.broadcast(qml.Hadamard, wires=[i for i in range(n_bits)], pattern="single")
    ## apply the oracle
    oracle = oracle_matrix(combo)
    qml.QubitUnitary(oracle,wires=[i for i in range(n_bits)])
    return qml.state()

**Codercise G.1.2.**
(a) Define the diffusion operator as a matrix, and visualize its effect on the amplitudes in the post-oracle state. We'll plot the amplitudes for combo=[0, 0, 0, 0]. Note that 'oracle_matrix(combo)' remains available.Hint.
Hint. The diffusion matrix can be written  $D = \frac{1}{2^{n-1}}1-I$,
where $1$ is the $2^n x 2^n$ matrix with $1$ in each entry. You will therefore find `np.ones()` useful in addition to `np.eye()`.

In [4]:
n_bits = 4

def diffusion_matrix():
    """Return the diffusion matrix.

    Returns:
        array[float]: The matrix representation of the diffusion operator.
    """
    size = 2**n_bits
    I = np.eye(size)
    fraction = 1/(2**(n_bits-1))
    diffusion = fraction * np.ones(shape = size)
    diffusion = diffusion - I
    return diffusion

@qml.qnode(dev)
def difforacle_amp(combo):
    """Apply the oracle and diffusion matrix to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle
        and diffusion.
    """
    ## superposition
    qml.broadcast(qml.Hadamard, wires=[i for i in range(n_bits)], pattern="single")
    ## apply the oracle
    oracle = oracle_matrix(combo)
    qml.QubitUnitary(oracle,wires=[i for i in range(n_bits)])
    diffusion = diffusion_matrix()
    qml.QubitUnitary(diffusion,wires=[i for i in range(n_bits)])
    return qml.state()

(b) Complete the code below and verify that after two Grover iterations, the amplitude for the solution state is amplified. The functions `oracle_matrix(combo)` and `diffusion_matrix()` remain available.

In [ ]:
@qml.qnode(dev)
def two_difforacle_amp(combo):
    """Apply the Grover operator twice to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The resulting quantum state.
    """
    qml.broadcast(qml.Hadamard, wires=[i for i in range(n_bits)], pattern="single")

    oracle = oracle_matrix(combo)
    diffusion = diffusion_matrix()

    for i in range(2):
        qml.QubitUnitary(oracle,wires=[i for i in range(n_bits)])
        qml.QubitUnitary(diffusion,wires=[i for i in range(n_bits)])

    return qml.state()
